Prior Setup


In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
!pip uninstall numpy
!pip install --upgrade --force-reinstall gensim numpy

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e71.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? 

Pull dataset

In [ ]:
import pandas as pd
import numpy as np

# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'hau/train-00000-of-00001.parquet', 'dev': 'hau/dev-00000-of-00001.parquet', 'test': 'hau/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/brighter-dataset/BRIGHTER-emotion-categories/" + splits["train"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Drop duplicates and rows with missing values

In [ ]:
df = df.applymap(lambda x: tuple(x) if isinstance(x, list) or isinstance(x, np.ndarray) else x)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.head()

<ipython-input-3-09257dd3d4a9>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: tuple(x) if isinstance(x, list) or isinstance(x, np.ndarray) else x)


,id,text,anger,disgust,fear,joy,sadness,surprise,emotions
0,hau_train_track_a_00001,kotu ta yi hukunci kan shari'ar zaben dan maja...,0,0,0,0,0,1,"(surprise,)"
1,hau_train_track_a_00002,"toh fah inji 'yan magana suka ce """"""""ana wata ...",0,0,0,0,0,1,"(surprise,)"
2,hau_train_track_a_00003,bincike ya nuna yan najeriya sun fi damuwa da ...,0,0,1,0,1,0,"(fear, sadness)"
3,hau_train_track_a_00004,kwamishina ya musanta rahoton masari ya cire k...,0,0,0,0,0,0,()
4,hau_train_track_a_00005,innalillahi wa inna ilaihir raji'un: allah ya ...,0,0,0,0,1,0,"(sadness,)"


Tokenize

In [ ]:
from nltk.tokenize import word_tokenize
df['tokens'] = df['text'].apply(word_tokenize)
df.head()

,id,text,anger,disgust,fear,joy,sadness,surprise,emotions,tokens
0,hau_train_track_a_00001,kotu ta yi hukunci kan shari'ar zaben dan maja...,0,0,0,0,0,1,"(surprise,)","[kotu, ta, yi, hukunci, kan, shari'ar, zaben, ..."
1,hau_train_track_a_00002,"toh fah inji 'yan magana suka ce """"""""ana wata ...",0,0,0,0,0,1,"(surprise,)","[toh, fah, inji, 'yan, magana, suka, ce, ``, '..."
2,hau_train_track_a_00003,bincike ya nuna yan najeriya sun fi damuwa da ...,0,0,1,0,1,0,"(fear, sadness)","[bincike, ya, nuna, yan, najeriya, sun, fi, da..."
3,hau_train_track_a_00004,kwamishina ya musanta rahoton masari ya cire k...,0,0,0,0,0,0,(),"[kwamishina, ya, musanta, rahoton, masari, ya,..."
4,hau_train_track_a_00005,innalillahi wa inna ilaihir raji'un: allah ya ...,0,0,0,0,1,0,"(sadness,)","[innalillahi, wa, inna, ilaihir, raji'un, :, a..."


Remove Stopwords

In [ ]:
# Import stopwords from https://github.com/stopwords-iso/stopwords-ha
import requests
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ha/master/stopwords-ha.txt"
r = requests.get(url)
stopwords = r.text.split("\n")
# remove stopwords from tokens column in df
df['tokens'] = df['tokens'].apply(lambda x: [item for item in x if item not in stopwords])
df.head()


Lowercase, remove punctuation, special characters (emojis too) and numerical tokens, ten lemmatize

In [ ]:
import re
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords, strip_punctuation, strip_numeric

# There is no direct PorterStemmer in recent Gensim versions.
# Using NLTK's PorterStemmer as an alternative.
import nltk
# You only need to download 'porter_stemmer' once
# nltk.download('porter_stemmer')
from nltk.stem.porter import PorterStemmer

import re
import string

def custom_strip_punctuation(text, chars_to_remove=None):
    if chars_to_remove is None:
        chars_to_remove = ',.!?;:()[]{}"\''

    translation_table = str.maketrans('', '', chars_to_remove)

    return text.translate(translation_table)

def preprocess_and_stem_tokens(tokens):
    processed_tokens = []
    stemmer = PorterStemmer()
    for token in tokens:
        # Convert token to string and lowercase
        token = str(token).lower()

        # Remove emojis (using a common regex pattern)
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U000025FF"  # geometric shapes
                               u"\U00002700-\U000027BF"  # dingbats
                               u"\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
                               u"\U0001FA00-\U0001FA6F"  # chess symbols
                               u"\U0001FA70-\U0001FAFF"  # symbols and pictographs extended-A
                               u"\U000024C2-\U0001F251"  # Enclosed characters
                               "]+", flags=re.UNICODE)
        token = emoji_pattern.sub(r'', token)

        # Remove punctuation
        token = custom_strip_punctuation(token, chars_to_remove=',"`[]{};')
        # Remove numerical tokens
        token = strip_numeric(token)

        # Only attempt stemming if the token is not empty after cleaning
        if token:
            # Stem the token
            processed_token = stemmer.stem(token)
            processed_tokens.append(processed_token)
    return processed_tokens

# Apply the function to the 'tokens' column
df['stemmed_tokens'] = df['tokens'].apply(preprocess_and_stem_tokens)
df.head()

,id,text,anger,disgust,fear,joy,sadness,surprise,emotions,tokens,stemmed_tokens
0,hau_train_track_a_00001,kotu ta yi hukunci kan shari'ar zaben dan maja...,0,0,0,0,0,1,"(surprise,)","[kotu, ta, yi, hukunci, kan, shari'ar, zaben, ...","[kotu, ta, yi, hukunci, kan, shari'ar, zaben, ..."
1,hau_train_track_a_00002,"toh fah inji 'yan magana suka ce """"""""ana wata ...",0,0,0,0,0,1,"(surprise,)","[toh, fah, inji, 'yan, magana, suka, ce, ``, '...","[toh, fah, inji, 'yan, magana, suka, ce, ``, '..."
2,hau_train_track_a_00003,bincike ya nuna yan najeriya sun fi damuwa da ...,0,0,1,0,1,0,"(fear, sadness)","[bincike, ya, nuna, yan, najeriya, sun, fi, da...","[bincik, ya, nuna, yan, najeriya, sun, fi, dam..."
3,hau_train_track_a_00004,kwamishina ya musanta rahoton masari ya cire k...,0,0,0,0,0,0,(),"[kwamishina, ya, musanta, rahoton, masari, ya,...","[kwamishina, ya, musanta, rahoton, masari, ya,..."
4,hau_train_track_a_00005,innalillahi wa inna ilaihir raji'un: allah ya ...,0,0,0,0,1,0,"(sadness,)","[innalillahi, wa, inna, ilaihir, raji'un, :, a...","[innalillahi, wa, inna, ilaihir, raji'un, :, a..."


Save file for reuse

In [ ]:
# Save file for reuse
df.to_csv('processed_data_it3.csv', index=False)